In [1]:
import pandas as pd
import numpy as np
import json
import pickle

In [6]:
train_predicted_file_path = "train_df_predicted.pkl"

In [7]:
df = pickle.load(open(train_predicted_file_path, 'rb'))
df.head()

,file_id,toc_section,toc_section_pos,toc_section_len,is_section_in_summary,toc_section_cleaned,pred,False,True
0,14755,Notes relating to the Company,6936,364,0,note relat compani,0,0.973294,0.026706
1,14611,Corporate governance,2658,686,0,corpor govern,0,0.983541,0.016459
2,6085,Energy T echnology,697,146,1,energi echnolog,0,0.743222,0.256778
3,18872,Areas of exploration,37,8,1,area explor,0,0.729578,0.270422
4,11699,"Segments, Products and Services",84,32,0,segment product servic,0,0.681333,0.318667


In [9]:
file_id_true_score_sum = df.groupby('file_id').sum()['True']
df['normalized_true_score'] = 0.0
for index, row in df.iterrows():
    try:
        key = row['file_id']
        df.at[index,'normalized_true_score']=row['True']/file_id_true_score_sum[key]
    except Exception as e:
        print(e)
        pass

In [12]:
final_scores = df.normalized_true_score
final_scores

0        0.010972
1        0.004532
2        0.091657
3        0.097262
4        0.096940
           ...   
54309    0.006226
54310    0.030609
54311    0.311790
54312    0.002666
54313    0.001883
Name: normalized_true_score, Length: 54314, dtype: float64

In [13]:
toc_sections = df.toc_section
section_scores ={}
for i, section in enumerate(toc_sections):
    if section in section_scores.keys():
        section_scores[section] = section_scores[section] + final_scores[i]
    else:
        section_scores[section] = final_scores[i]
section_scores = dict(sorted(section_scores.items(), key=lambda item: item[1], reverse=True))
section_scores

{'Chairman’s Statement': 258.3235409011416,
 'Chairman’s statement': 248.38691905759592,
 'Highlights': 204.8765366957901,
 'Chief Executive’s Review': 117.71206133554467,
 'Chief Executive’s review': 104.32181892450329,
 'start': 93.94581977437879,
 'Financial Highlights': 59.09129171046692,
 'Financial highlights': 44.45097748380377,
 'Chief Executive’s Statement': 38.947681489020404,
 'Chief Executive’s Report': 32.26688617683165,
 'Chief Executive’s statement': 31.808760766320354,
 'At a glance': 29.99457272345307,
 'Chief Executive Officer’s review': 22.475189703355284,
 'Group at a glance': 20.91304146122764,
 'Chief executive’s review': 19.607252912628233,
 'CHAIRMAN’S STATEMENT': 18.05381695706053,
 'Chief Executive Officer’s Review': 16.648376162467038,
 'Our strategy': 16.62203005676601,
 'At a Glance': 15.234880334683263,
 'Group at a Glance': 14.992335283785062,
 'Chief Executive Officer’s Statement': 11.697359669602271,
 'Notice of Annual General Meeting': 11.4188237528509

In [14]:
top_50_sections = list(section_scores.items())[:50]
top_50_sections

[('Chairman’s Statement', 258.3235409011416),
 ('Chairman’s statement', 248.38691905759592),
 ('Highlights', 204.8765366957901),
 ('Chief Executive’s Review', 117.71206133554467),
 ('Chief Executive’s review', 104.32181892450329),
 ('start', 93.94581977437879),
 ('Financial Highlights', 59.09129171046692),
 ('Financial highlights', 44.45097748380377),
 ('Chief Executive’s Statement', 38.947681489020404),
 ('Chief Executive’s Report', 32.26688617683165),
 ('Chief Executive’s statement', 31.808760766320354),
 ('At a glance', 29.99457272345307),
 ('Chief Executive Officer’s review', 22.475189703355284),
 ('Group at a glance', 20.91304146122764),
 ('Chief executive’s review', 19.607252912628233),
 ('CHAIRMAN’S STATEMENT', 18.05381695706053),
 ('Chief Executive Officer’s Review', 16.648376162467038),
 ('Our strategy', 16.62203005676601),
 ('At a Glance', 15.234880334683263),
 ('Group at a Glance', 14.992335283785062),
 ('Chief Executive Officer’s Statement', 11.697359669602271),
 ('Notice o

In [15]:
top_50_sections_df = pd.DataFrame(columns=['section_name', 'score'])

In [16]:
for section in top_50_sections:
    top_50_sections_df.at[len(top_50_sections_df)] = [section[0], section[1]]

In [17]:
top_50_sections_df

,section_name,score
0,Chairman’s Statement,258.323541
1,Chairman’s statement,248.386919
2,Highlights,204.876537
3,Chief Executive’s Review,117.712061
4,Chief Executive’s review,104.321819
5,start,93.94582
6,Financial Highlights,59.091292
7,Financial highlights,44.450977
8,Chief Executive’s Statement,38.947681
9,Chief Executive’s Report,32.266886


In [18]:
top_50_sections_df.to_csv('top_50_sections_scores_original_toc_sections.csv', index=False)